In [190]:
import numpy as np
import pandas as pd
import datetime
from dateutil.parser import parse
import requests, re, time
import pandas_datareader
import pickle
import math

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import seaborn as sns 

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import linearmodels as ln
from pystout import pystout



import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from selenium import webdriver
import time

import geopy.distance

# Regressions behind Appendix A: Comparing Cityring-events

## EVENT: CITYRING-PLANS ANNOUNCED (2005)

In [191]:
# Loading and creating base-data:
Apartments = pickle.load(open('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Apartment Data API.pkl','rb'))
Apartments['Year Sold'] = [i.year for i in Apartments['Date_sold']]
Apartments['Closest Metro dist'] = [i['Distance'] for i in Apartments['Closest Metro']]
Apartments['Cityring'] = [i['Cityring'] for i in Apartments['Closest Metro']]
Apartments['log_price_sq_m_1000'] = np.log(Apartments['Price_sq_m_1000'])
# Generating Close-dummy:
Apartments['Close to Metro'] = 0
for i in range(0,len(Apartments)):
    if Apartments['Closest Metro dist'][i] < 0.5:
        Apartments['Close to Metro'][i] = 1

# Picking out those apartments having the cityring as the closest metro:
Cityring = Apartments[(Apartments['Cityring'] == 1)]
Cityring.reset_index(inplace=True)

In [192]:
# Creating Event-dataframe of the two tested years:
Cityring_04 = Cityring[(Cityring['Year Sold'] == 2004)]
Cityring_06 = Cityring[(Cityring['Year Sold'] == 2006)]

Event = pd.concat([Cityring_04,Cityring_06],ignore_index=True)

In [193]:
# Generating Announcement-Dummy:
Event['Announced'] = 0
for i in range(0,len(Event)):
    if Event['Year Sold'][i] == 2006:
        Event['Announced'][i] = 1

In [194]:
# Generating location dummies and deleting reference-area (those not being defined):
area_dummy = pd.get_dummies(Event['Area'])
del area_dummy['NaN']

In [195]:
# Generating dataframe to be regressed on:
X = pd.DataFrame()
X['Rooms'] = Event['Rooms']
X['Close to Metro'] = Event['Close to Metro']
X['Event'] = Event['Announced']
X['EventXClose'] = Event['Close to Metro'] * Event['Announced']
X = pd.merge(X,area_dummy,left_index=True,right_index=True)

In [196]:
X = sm.add_constant(X)
model = sm.OLS(Event['log_price_sq_m_1000'],X)
Results_Announced = model.fit(cov_type='HC1')
print(Results_Announced.summary())

OLS Regression Results                            
Dep. Variable:     log_price_sq_m_1000   R-squared:                       0.263
Model:                             OLS   Adj. R-squared:                  0.255
Method:                  Least Squares   F-statistic:                     43.48
Date:                 Wed, 05 Aug 2020   Prob (F-statistic):           2.52e-70
Time:                         09:52:15   Log-Likelihood:                -411.95
No. Observations:                  902   AIC:                             845.9
Df Residuals:                      891   BIC:                             898.7
Df Model:                           10                                         
Covariance Type:                   HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              3.1346      0.055     57.321      0.000     

## EVENT: Cityring-Law passed in Parliament (2007)

In [197]:
C2006 = Cityring[(Cityring['Year Sold'] == 2006)]
C2008 = Cityring[(Cityring['Year Sold'] == 2008)]

Event = pd.concat([C2006,C2008],ignore_index=True)

In [198]:
Event['Law Passed'] = 0
for i in range(0,len(Event)):
    if Event['Year Sold'][i] == 2008:
        Event['Law Passed'][i] = 1

In [199]:
# Generating location dummies:
Event['Metro_loc'] = [i['Metro'] for i in Event['Closest Metro']]
area_dummy = pd.get_dummies(Event['Area'])
del area_dummy['NaN']

In [200]:
X = pd.DataFrame()
X['Rooms'] = Event['Rooms']
X['Close to Metro'] = Event['Close to Metro']
X['Event'] = Event['Law Passed']
X['EventXClose'] = Event['Close to Metro'] * Event['Law Passed']
X = pd.merge(X,area_dummy,left_index=True,right_index=True)

In [201]:
X = sm.add_constant(X)
model = sm.OLS(Event['log_price_sq_m_1000'],X)
Results_Passed = model.fit(cov_type='HC1')
print(Results_Passed.summary())

OLS Regression Results                            
Dep. Variable:     log_price_sq_m_1000   R-squared:                       0.121
Model:                             OLS   Adj. R-squared:                  0.110
Method:                  Least Squares   F-statistic:                     15.68
Date:                 Wed, 05 Aug 2020   Prob (F-statistic):           1.37e-23
Time:                         09:52:15   Log-Likelihood:                -381.07
No. Observations:                  746   AIC:                             782.1
Df Residuals:                      736   BIC:                             828.3
Df Model:                            9                                         
Covariance Type:                   HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              3.3863      0.079     43.033      0.000     

## EVENT: CONSTRUCTION BEGAN

In [202]:
C2008 = Cityring[(Cityring['Year Sold'] == 2008)]
C2010 = Cityring[(Cityring['Year Sold'] == 2010)]

Event = pd.concat([C2008,C2010],ignore_index=True)

In [203]:
Event['Construction'] = 0
for i in range(0,len(Event)):
    if Event['Year Sold'][i] == 2010:
        Event['Construction'][i] = 1

In [204]:
# Generating location dummies:
area_dummy = pd.get_dummies(Event['Area'])
del area_dummy['NaN']

In [205]:
X = pd.DataFrame()
X['Rooms'] = Event['Rooms']
X['Close to Metro'] = Event['Close to Metro']
X['Event'] = Event['Construction']
X['EventXClose'] = Event['Close to Metro'] * Event['Construction']
X = pd.merge(X,area_dummy,left_index=True,right_index=True)

In [206]:
X = sm.add_constant(X)
model = sm.OLS(Event['log_price_sq_m_1000'],X)
Results_Construction = model.fit(cov_type='HC1')
print(Results_Construction.summary())

OLS Regression Results                            
Dep. Variable:     log_price_sq_m_1000   R-squared:                       0.090
Model:                             OLS   Adj. R-squared:                  0.079
Method:                  Least Squares   F-statistic:                     9.420
Date:                 Wed, 05 Aug 2020   Prob (F-statistic):           1.27e-13
Time:                         09:52:15   Log-Likelihood:                -478.95
No. Observations:                  727   AIC:                             977.9
Df Residuals:                      717   BIC:                             1024.
Df Model:                            9                                         
Covariance Type:                   HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              2.8527      0.141     20.225      0.000     

## EVENT: BUILDING FINISHED (2019)

In [207]:
C2018 = Cityring[(Cityring['Year Sold'] == 2018)]
C2020 = Cityring[(Cityring['Year Sold'] == 2020)]

In [208]:
Event = pd.concat([C2018,C2020],ignore_index=True)

In [209]:
Event['Opened'] = 0
for i in range(0,len(Event)):
    if Event['Year Sold'][i] == 2020:
        Event['Opened'][i] = 1

In [210]:
# Generating location dummies:
area_dummy = pd.get_dummies(Event['Area'])
del area_dummy['NaN']

In [211]:
X = pd.DataFrame()
X['Rooms'] = Event['Rooms']
X['Close to Metro'] = Event['Close to Metro']
X['Event'] = Event['Opened']
X['EventXClose'] = Event['Close to Metro'] * Event['Opened']
X = pd.merge(X,area_dummy,left_index=True,right_index=True)

In [212]:
X = sm.add_constant(X)
model = sm.OLS(Event['log_price_sq_m_1000'],X)
Results_Opening = model.fit(cov_type='HC1')
print(Results_Opening.summary())

OLS Regression Results                            
Dep. Variable:     log_price_sq_m_1000   R-squared:                       0.028
Model:                             OLS   Adj. R-squared:                  0.026
Method:                  Least Squares   F-statistic:                     15.98
Date:                 Wed, 05 Aug 2020   Prob (F-statistic):           1.12e-28
Time:                         09:52:17   Log-Likelihood:                -2700.8
No. Observations:                 4950   AIC:                             5424.
Df Residuals:                     4939   BIC:                             5495.
Df Model:                           10                                         
Covariance Type:                   HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              3.5017      0.035    100.298      0.000     

In [213]:
pystout(models=[Results_Announced,Results_Passed,Results_Construction,Results_Opening],
        file='/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Tex files/Metro Regressions 3.tex',
        digits=3,
        addnotes=['(*): ','(**)','(***)'],
        endog_names=['Announcement','Law passed','Construction Began','Opened'],
        varlabels={'const':'Constant','displacement':'Disp','mpg':'MPG'},
        mgroups={'y = log(Price per sq. m, DKK)':[1,3]},
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}','fvalue':'F-stat'}
        )